In [1]:
import os
import json
from dotenv import load_dotenv
from swarm import Swarm, Agent
import openai
from prompts import *

from Chat import AgentClient
from Toolbox import ImageProcessingToolBoxes

# Set the environment variable for the OpenAI API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

In [13]:
test_image_path = './test-images/a0013-MB_20030906_001.jpg'
output_dir_path = f'./test-output/{os.path.splitext(os.path.basename(test_image_path))[0]}'
image_process = ImageProcessingToolBoxes(image_path=test_image_path, output_dir_name=output_dir_path, debug=True, save_high_resolution=False)
chat_client = AgentClient(api_key=api_key, toolbox_instance=image_process, debug=False)
global_style = None

In [14]:
#给每一步编号，并回到某一步

In [15]:
_ = chat_client.agent_interaction(
    system_prompt=image_content_analyzer_prompt()
)
global_style = chat_client.agent_interaction(
    system_prompt=image_global_style_analyzer_prompt()
)
_ = chat_client.agent_interaction(
    system_prompt=global_retouching_concept_prompt(function_aspects=image_process.get_function_short_description(), global_instruction=global_style)
)
_ = chat_client.agent_get_plan(
    system_prompt=retouching_planing_system_prompt(),
    user_prompt=retouching_planing_user_prompt(image_process.get_function_names(), global_instruction=global_style)
)

while image_process.processing_plan:
    current_function = image_process.processing_plan[0]

    if image_process.plan_status != current_function:  # 如果正在执行的function和现在的一样，说明是重新执行的。否则是重新开始
        image_process.set_plan_status(current_function)
        image_process.set_satisfactory_status(False)  # 开始新的function默认不满意

        chat_client.agent_execute_plan(
            system_prompt=retouching_execute_system_prompt(),
            user_prompt=retouching_execute_user_prompt(current_function),
            current_function_name=current_function)
    else:
        chat_client.agent_execute_plan(
            system_prompt=retouching_execute_system_prompt(),
            user_prompt=retouching_re_execute_user_prompt(current_function),
            current_function_name=current_function)

    chat_client.agent_reflection_plan(
        system_prompt=retouching_reflection_system_prompt(),
        combine_image=False
    )

    if image_process.satisfactory_status:
        image_process.finish_current_plan()



`func_to_return_responses`, response: 

### Analysis of Main Content
The photo features a woman standing under a portable canopy, holding an orange foam noodle and some tape. The setting appears to be a sunny outdoor area, possibly a park or backyard, with greenery in the background and a small paved area beneath her. She's casually dressed in a loose-fitting pink shirt. The image is likely candid, capturing her in action, perhaps preparing for an outdoor event or activity.

### Evaluation of Photographic Value
The image is shot at eye level, which provides a natural view of the subject. The lighting is bright and sunny, casting clear shadows on the ground, which adds depth to the photo. The use of natural light enhances the colors, making them vibrant but also causing some harsh shadows and highlights. The composition is straightforward, focusing primarily on the subject, with the canopy framing her and serving as an informal guide for the viewer's eye.

### Speculation on Emotions 

In [16]:
image_process.check_history_messages()


Message 1 | Role: assistant
--------------------------------------------------------------------------------
### Analysis of Main Content
The photo features a woman standing under a portable canopy, holding an orange foam noodle and some tape. The setting appears to be a sunny outdoor area, possibly a park or backyard, with greenery in the background and a small paved area beneath her. She's casually dressed in a loose-fitting pink shirt. The image is likely candid, capturing her in action, perhaps preparing for an outdoor event or activity.

### Evaluation of Photographic Value
The image is shot at eye level, which provides a natural view of the subject. The lighting is bright and sunny, casting clear shadows on the ground, which adds depth to the photo. The use of natural light enhances the colors, making them vibrant but also causing some harsh shadows and highlights. The composition is straightforward, focusing primarily on the subject, with the canopy framing her and serving as a

In [17]:
sum(chat_client.total_tokens)

67439

In [5]:
client = openai.OpenAI(api_key=api_key)

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    }
                },
                "required": ["order_id"],
                "additionalProperties": False
            }
        }
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})
messages.append({"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"})
messages.append({"role": "user", "content": "i think it is order_12345"})

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    tools=tools
)

In [8]:
response.usage.total_tokens

159